# T81 Notebook 02 — VM Trace Walkthrough

This notebook provides a **conceptual trace** of a tiny T81-style virtual machine.

Goals:

1. Show how a small instruction stream is fetched, decoded, and executed.
2. Visualize **register state** and **program counter** over time.
3. Connect the toy model to the real T81 VM (HanoiVM lineage) and the TISC spec.

> This is **not** the actual T81 VM implementation.  
> It is a deliberately simplified, pure-Python model to help you reason about control flow and state.


## 0. A Minimal Instruction Set

We define a small, TISC-inspired instruction set:

- `LOADI r, imm`   — load an immediate integer into register `r`.
- `ADD   rd, ra, rb` — `rd = ra + rb`.
- `SUB   rd, ra, rb` — `rd = ra - rb`.
- `JNZ   r, offset`  — if register `r` != 0, jump relative by `offset`.
- `HALT`            — stop execution.

Registers are modeled as simple Python integers.
A real T81 VM would use T81 types and observe strict determinism and spec semantics.


In [ ]:
from dataclasses import dataclass
from typing import List, Optional, Dict, Any

@dataclass
class Instruction:
    op: str
    args: tuple

class ToyVM:
    def __init__(self, num_registers: int = 4):
        self.registers = [0] * num_registers
        self.pc = 0
        self.program: List[Instruction] = []
        self.halted = False
        self.trace: List[Dict[str, Any]] = []

    def load_program(self, program: List[Instruction]):
        self.program = program
        self.pc = 0
        self.halted = False
        self.trace.clear()

    def step(self):
        if self.halted:
            return
        if not (0 <= self.pc < len(self.program)):
            raise RuntimeError(f"PC out of range: {self.pc}")

        instr = self.program[self.pc]
        op, args = instr.op, instr.args

        # Record state before executing the instruction
        snapshot = {
            "pc": self.pc,
            "op": op,
            "args": args,
            "registers_before": self.registers.copy(),
        }

        self.pc += 1  # default: move to next instruction

        if op == "LOADI":
            r, imm = args
            self.registers[r] = imm
        elif op == "ADD":
            rd, ra, rb = args
            self.registers[rd] = self.registers[ra] + self.registers[rb]
        elif op == "SUB":
            rd, ra, rb = args
            self.registers[rd] = self.registers[ra] - self.registers[rb]
        elif op == "JNZ":
            r, offset = args
            if self.registers[r] != 0:
                self.pc += offset
        elif op == "HALT":
            self.halted = True
        else:
            raise ValueError(f"Unknown opcode: {op}")

        snapshot["registers_after"] = self.registers.copy()
        snapshot["halted"] = self.halted
        snapshot["next_pc"] = self.pc
        self.trace.append(snapshot)

    def run(self, max_steps: Optional[int] = 100):
        steps = 0
        while not self.halted and steps < max_steps:
            self.step()
            steps += 1
        if steps >= max_steps and not self.halted:
            raise RuntimeError("ToyVM: exceeded max_steps without HALT.")

def print_trace(vm: ToyVM):
    print("PC | OP      | ARGS      | REGISTERS BEFORE   | REGISTERS AFTER    | HALT | NEXT_PC")
    print("-" * 90)
    for event in vm.trace:
        pc = event["pc"]
        op = event["op"]
        args = event["args"]
        rb = event["registers_before"]
        ra = event["registers_after"]
        halted = event["halted"]
        npc = event["next_pc"]
        print(f"{pc:2d} | {op:<7s} | {str(args):<9s} | {str(rb):<18s} | {str(ra):<18s} | {halted!s:^5s} | {npc:7d}")


## 1. Example Program: Sum of First N Integers

We build a simple program in this toy ISA to compute the sum:

\( S = 1 + 2 + … + N \)

Pseudo-code:

```text
r0 = N
r1 = 0      # accumulator

loop:
  r1 = r1 + r0
  r0 = r0 - 1
  if r0 != 0: jump to loop
halt
```

We use `JNZ` to implement the loop.


In [ ]:
# Build the program: sum of 1..N where N is given as an immediate.
N = 5

program = [
    Instruction("LOADI", (0, N)),      # r0 = N
    Instruction("LOADI", (1, 0)),      # r1 = 0 (accumulator)
    # loop_start at pc = 2
    Instruction("ADD",   (1, 1, 0)),  # r1 = r1 + r0
    Instruction("SUB",   (0, 0, 2)),  # r0 = r0 - r2 (we'll set r2 = 1)
    Instruction("JNZ",   (0, -2)),    # if r0 != 0, jump back to ADD
    Instruction("HALT",  ()),         # stop
]

# We need r2 = 1 for the decrement.
# Insert that initialization after r1 = 0.
program.insert(2, Instruction("LOADI", (2, 1)))  # pc=2

vm = ToyVM(num_registers=4)
vm.load_program(program)
vm.run()
print_trace(vm)
print("\nFinal registers:", vm.registers)
expected = N * (N + 1) // 2
print(f"Expected sum S = {expected}")


## 2. Relating the Toy VM to T81’s Real VM

In the actual T81 stack:

- The instruction set is defined in the **TISC** specification (see `spec/tisc-spec.md` or equivalent).
- The VM is implemented in C++ and/or preserved in legacy CWEB sources under `legacy/hanoivm/`.
- Registers and memory use T81-style numeric types, not plain Python ints.

This toy model is useful for:

- Getting a feel for **PC-relative jumps** and control flow.
- Experimenting with small examples before you dive into the real VM.
- Teaching / documenting how instruction traces correspond to high-level behavior.


## 3. Exercises / Extensions

If you want to explore further:

1. Extend the toy VM with a `MUL` instruction and implement a small factorial program.
2. Change registers to hold toy balanced ternary values (e.g., using helpers from notebook 01).
3. Add a simple **tracing flag** that stops execution after a given number of steps and prints a diagnostic.
4. Compare the control flow of this toy program to a real T81Lang snippet that compiles down to a similar loop.

For real-world work, use:

- `spec/t81vm-spec.md` and `spec/tisc-spec.md` for authoritative instruction semantics.
- The C++ headers and source files under `include/t81/vm/` and `src/`.
- The existing tests in the repository that exercise the VM and TISC pipeline.
